## 시계열 다루기

Pandas는 금융 모델링을 위해 개발돼 예상대로 날짜, 시간, 시간 인덱스를 가진 데이터를 다루는 매우 다양한 도구를 가지고 있음

날짜와 시간 데이터의 종류
+ ``타임스태프`` : 특정 시점 (예, 2015년 6월 4일 오전 7시)
+ ``시간 간격``과 ``기간`` : 특정 시작점과 종료점 사이의 시간의 길이 (예:2015년 or 하루를 구성하는 24시간)
+ ``시간 델타`` 나 ``지속 기간`` : 정확한 시간 길이 (예: 22.56초의 시간)

### 기본 파이썬 날짜와 시간 : datetime & dateutil

제3자 모듈인 dateutil과 함께 dateitme 모듈을 사용해 날짜와 시간에 여러 가지 유용한 기능을 신속하게 수행할 수 있음

In [1]:
#datetime 타입을 사용해 날짜를 직접 구성
from datetime import datetime
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

In [ ]:
#dateutil 을 이용해 다양한 문자열 형태로부터 날짜를 해석
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

In [3]:
#datetime 객체를 갖고 나면 요일을 출력하는 등의 작업 가능
date.strftime('%A')

'Saturday'

### 타입이 지정된 시간 배열 : NumPy의 datetime64

datetime64 dtype은 날짜를 64비트 정수로 인코딩해 날짜 배열ㅇ르 매우 간결하게 표현하게 해준다. datetime64는 매우 구체적인 입력 형식이 필요.

In [9]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

In [10]:
#날짜 포맷이 정해지고 나면 거기에 벡터화된 연산을 빠르게 수행 가능
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

In [11]:
#하루 단위의 datetime
np.datetime64('2015-07-04')

np.datetime64('2015-07-04')

In [12]:
#분 단위의 datetime
np.datetime64('2015-07-04 12:00')

np.datetime64('2015-07-04T12:00')

In [13]:
#다양한 포맷 코드 중 하나를 사용해 원하는 기본 단위를 정할 수 있음
#예를 들면, 다음은 시간의 단위를 나노초로 정한 것
np.datetime64('2015-07-04 12:59:59.50', 'ns')

np.datetime64('2015-07-04T12:59:59.500000000')